In [1]:
import numpy as np
import matplotlib.pyplot as pyplot
import pandas as pd
import scipy.sparse as sps
%matplotlib inline
%load_ext Cython

from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
from Evaluation.Evaluator import EvaluatorHoldout
from Recommenders.MatrixFactorization.PureSVDRecommender import PureSVDRecommender
import optuna

C:\Users\feder\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# data_train_path="/kaggle/input/recommender-system-2023-challenge-polimi/data_train.csv"
# data_target_user_path="/kaggle/input/recommender-system-2023-challenge-polimi/data_target_users_test.csv"
data_train_path="data_train.csv"
data_target_user_path="data_target_users_test.csv"
data_train = pd.read_csv(data_train_path)
data_target = pd.read_csv(data_target_user_path)

In [3]:
URM_all =  data_train.pivot(index='row', columns='col', values='data').fillna(0)
item_map = {i : item for i, item in enumerate(URM_all.columns)}
user_map = {i : user for i, user in enumerate(data_target["user_id"])}
item_map_inv = {item : i for i, item in item_map.items()}
user_map_inv = {user : i for i, user in user_map.items()}
missing_index = [x for x in range(1,13025) if x not in URM_all.index.tolist()]
add_urm = pd.DataFrame(index = missing_index, columns = URM_all.columns).fillna(0)
URM_all = pd.concat([URM_all, add_urm]).sort_index()
del add_urm
del missing_index
#data_target["user_id"] = data_target["user_id"]
URM_all = URM_all.to_numpy()
URM_all = sps.csr_matrix(URM_all)
URM_all

<13024x22222 sparse matrix of type '<class 'numpy.float64'>'
	with 478730 stored elements in Compressed Sparse Row format>

In [4]:
test_folds = 3

URM_trains = [None] * test_folds
evaluator_tests = [None] * test_folds
for i in range(0, test_folds):
    URM_trains[i], URM_test = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.80)
    evaluator_tests[i] = EvaluatorHoldout(URM_test, cutoff_list=[10])

EvaluatorHoldout: Ignoring 2494 (19.1%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 2564 (19.7%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 2552 (19.6%) Users that have less than 1 test interactions


In [9]:
def train_evaluate(optuna_trial):
    factors = optuna_trial.suggest_int("factors", 1, 1000)
    
    mAP = 0.0
    for i in range(0, test_folds):
        SLIM_recommender = PureSVDRecommender(URM_trains[i], verbose=False)
        SLIM_recommender.fit(num_factors=factors)
        result_df, _ = evaluator_tests[i].evaluateRecommender(SLIM_recommender)
        mAP += result_df["MAP"].values[0]
    mAP /= test_folds
    return mAP

In [10]:
study = optuna.create_study(direction="maximize")
study.optimize(train_evaluate, n_trials=500)

[I 2023-12-11 21:55:11,738] A new study created in memory with name: no-name-b8603afe-ab9d-4356-8a8f-17e6c034849e


EvaluatorHoldout: Processed 10530 (100.0%) in 9.50 sec. Users per second: 1108
EvaluatorHoldout: Processed 10460 (100.0%) in 9.48 sec. Users per second: 1103
EvaluatorHoldout: Processed 10472 (100.0%) in 9.58 sec. Users per second: 1093


[I 2023-12-11 21:55:59,113] Trial 0 finished with value: 0.02956656536732009 and parameters: {'factors': 235}. Best is trial 0 with value: 0.02956656536732009.


EvaluatorHoldout: Processed 10530 (100.0%) in 10.56 sec. Users per second: 997
EvaluatorHoldout: Processed 10460 (100.0%) in 10.39 sec. Users per second: 1007
EvaluatorHoldout: Processed 10472 (100.0%) in 9.96 sec. Users per second: 1051


[I 2023-12-11 21:56:43,625] Trial 1 finished with value: 0.03069235079246697 and parameters: {'factors': 150}. Best is trial 1 with value: 0.03069235079246697.


...

EvaluatorHoldout: Processed 10530 (100.0%) in 11.13 sec. Users per second: 946
EvaluatorHoldout: Processed 10460 (100.0%) in 11.28 sec. Users per second: 928
EvaluatorHoldout: Processed 10472 (100.0%) in 11.04 sec. Users per second: 949


[I 2023-12-11 22:32:28,931] Trial 29 finished with value: 0.029730205310431912 and parameters: {'factors': 225}. Best is trial 19 with value: 0.03097119496956199.


EvaluatorHoldout: Processed 10530 (100.0%) in 10.51 sec. Users per second: 1002


[W 2023-12-11 22:32:49,487] Trial 30 failed with parameters: {'factors': 94} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "C:\Users\feder\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\optuna\study\_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "C:\Users\feder\AppData\Local\Temp\ipykernel_14484\358869383.py", line 8, in train_evaluate
    result_df, _ = evaluator_tests[i].evaluateRecommender(SLIM_recommender)
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\feder\Documents\DEV\my_RecSyS_Course_AT_PoliMi\Evaluation\Evaluator.py", line 276, in evaluateRecommender
    results_dict = self._run_evaluation_on_selected_users(recommender_object, self.users_to_evaluate)
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

KeyboardInterrupt: 